In [1]:
import geopandas
import networkx as nx
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os

import snman
from snman import osmnx_customized as oxc
from snman.constants import *

#PERIMETER = '_paper2_example'
PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
outputs_path_accessibility = os.path.join(data_directory, 'outputs', '_accessibility_debug')

CRS_internal = 2056      # for Zurich

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
population_sample = 0.05
statent_id = 68292494
distance_limit = 70*1000
destinations_sample = 0.05
cycling_speed_kmh = 18
walking_speed_kmh = 1.34 * 3.6
access_egress_detour_factor = 2
departure_time = datetime.datetime(2024, 2, 22, 18, 00)
min_travel_time = 5*60
min_euclidian_distance = 100

rebuild_inputs = True
STATE = 'after'
TIME = '18:00'

if STATE=='before':
    ln_desc = KEY_LANES_DESCRIPTION
else:
    ln_desc = KEY_LANES_DESCRIPTION_AFTER

In [3]:
if rebuild_inputs and 1:
    r5_transit_network = r5py.TransportNetwork(
        os.path.join(outputs_path_accessibility, 'before_oneway_links_default.osm.pbf'),
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip'),
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip'),
    )

In [4]:
# Load street graph
if rebuild_inputs and 1:
    G = snman.io.load_street_graph(
        os.path.join(outputs_path, 'G_edges.gpkg'),
        os.path.join(outputs_path, 'G_nodes.gpkg'),
        crs=CRS_internal
    )

In [5]:
if rebuild_inputs:
    G_modes = {}
    L_modes = {}

In [6]:
# create mode graphs for other modes
if rebuild_inputs:
    for mode in [MODE_CYCLING, MODE_FOOT]:
        print(f'Make street and lane graphs for {mode}')
        G_modes[mode] = copy.deepcopy(G)
        snman.street_graph.filter_lanes_by_modes(
            G_modes[mode], [mode], lane_description_key=ln_desc
        )
        L_modes[mode] = snman.lane_graph.create_lane_graph(
            G_modes[mode], lanes_attribute=ln_desc
        )

Make street and lane graphs for cycling
Make street and lane graphs for foot


In [7]:
if rebuild_inputs:
    # load the new car street graph created from MATSim output
    G_modes[MODE_PRIVATE_CARS] = snman.io.load_street_graph(
        os.path.join(outputs_path, f'tt_{STATE}_edges.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gpkg'),
        crs=CRS_internal,
        unstringify_attributes={'lanes': snman.space_allocation.space_allocation_from_string}
    )
    
    # we must avoid that the start/end point of trips are matched onto nodes that not accessible by car
    # therefore, we reduce the graph only to those links and nodes that are accessible by car
    snman.street_graph.filter_lanes_by_modes(
        G_modes[MODE_PRIVATE_CARS], [MODE_PRIVATE_CARS], lane_description_key='lanes'
    )

In [8]:
if rebuild_inputs:
    L_modes[MODE_PRIVATE_CARS] = snman.lane_graph.create_lane_graph(
        G_modes[MODE_PRIVATE_CARS], lanes_attribute='lanes', cast_attributes={'matsim_tt_cars': f'car_{TIME}'}
    )

In [9]:
if rebuild_inputs:
    # Import Statent grid
    statent = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
    )
    statent.rename(columns={'RELI': 'id'}, inplace=True)
    statent

In [10]:
if rebuild_inputs:
    # read statpop that has been pre-joined with statent
    statpop_with_statent_ids = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
    )
    
    statpop_with_statent_ids.head()

In [11]:
# filter statpop for residents within the given origin statent cell
statpop_filtered = statpop_with_statent_ids.query(f'statent_id == {statent_id}')
statpop_filtered = statpop_filtered.sample(frac=population_sample, random_state=0)
statpop_filtered

,statyear,yearofbirth,sex,maritalstatus,residencepermit,age,residentpermit,federalbuildingid,geometry,index_right,statent_id,distance_to_statent_point
record,,,,,,,,,,,,
7722724,2017,1977,1,1,-2,40,-2,899847315,POINT (2682915.001 1249401.001),28794.0,68292494.0,15.033988
4128950,2017,1965,2,4,301,52,3,899847319,POINT (2682892.001 1249431.001),28794.0,68292494.0,32.016570
8324543,2017,1984,2,1,-2,33,-2,899847305,POINT (2682938.001 1249356.001),28794.0,68292494.0,58.137308
5044070,2017,1957,2,2,301,60,3,899847315,POINT (2682915.001 1249401.001),28794.0,68292494.0,15.033988
7259080,2017,1978,2,1,-2,39,-2,899847305,POINT (2682938.001 1249356.001),28794.0,68292494.0,58.137308
9617550,2017,1972,2,1,-2,45,-2,899847318,POINT (2682891.001 1249447.001),28794.0,68292494.0,47.854948
5778759,2017,1979,1,2,-2,38,-2,899847313,POINT (2682938.001 1249402.001),28794.0,68292494.0,38.053271
11124682,2017,1942,1,2,-2,75,-2,899847318,POINT (2682891.001 1249447.001),28794.0,68292494.0,47.854948
3434419,2017,1983,2,1,201,34,2,899847316,POINT (2682894.001 1249408.001),28794.0,68292494.0,10.000541


In [12]:
# make a light version of statpop
statpop_reduced = statpop_filtered.reset_index()[[
    'record', 'age', 'sex', 'maritalstatus', 'residencepermit', 'residentpermit', 'statent_id', 'geometry'
]]
statpop_reduced.set_index('record', inplace=True)
statpop_reduced

,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry
record,,,,,,,
7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001)
4128950,52,2,4,301,3,68292494.0,POINT (2682892.001 1249431.001)
8324543,33,2,1,-2,-2,68292494.0,POINT (2682938.001 1249356.001)
5044070,60,2,2,301,3,68292494.0,POINT (2682915.001 1249401.001)
7259080,39,2,1,-2,-2,68292494.0,POINT (2682938.001 1249356.001)
9617550,45,2,1,-2,-2,68292494.0,POINT (2682891.001 1249447.001)
5778759,38,1,2,-2,-2,68292494.0,POINT (2682938.001 1249402.001)
11124682,75,1,2,-2,-2,68292494.0,POINT (2682891.001 1249447.001)
3434419,34,2,1,201,2,68292494.0,POINT (2682894.001 1249408.001)


In [13]:
# filter statent for cells within a distance limit from the origin
origin = statent.set_index('id').geometry[statent_id]
origins = gpd.GeoDataFrame(
    {
        "id": [statent_id],
        "geometry": [origin]
    },
    crs="EPSG:2056",
)
statent_filtered = statent[statent['geometry'].within(
    origin.buffer(distance_limit)
)]
statent_filtered = gpd.GeoDataFrame(statent_filtered)
statent_filtered

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR1,VOLLZEITAEQ_SEKTOR1_FRAUEN,VOLLZEITAEQ_SEKTOR1_MAENNER,VOLLZEITAEQ_SEKTOR2,VOLLZEITAEQ_SEKTOR2_FRAUEN,VOLLZEITAEQ_SEKTOR2_MAENNER,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry
0,2021,2023,2632200,1247300,63222473,4,0,4,4,7,...,0.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,0.0,POINT (2632200.000 1247300.000)
1,2021,2023,2632300,1248100,63232481,4,4,0,0,4,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2632300.000 1248100.000)
2,2021,2023,2632400,1247300,63242473,4,0,0,4,5,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632400.000 1247300.000)
3,2021,2023,2632400,1247500,63242475,4,0,4,0,4,...,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,POINT (2632400.000 1247500.000)
4,2021,2023,2632500,1247200,63252472,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632500.000 1247200.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58739,2021,2023,2728300,1222500,72832225,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728300.000 1222500.000)
58740,2021,2023,2728300,1222900,72832229,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,POINT (2728300.000 1222900.000)
58741,2021,2023,2728400,1222700,72842227,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728400.000 1222700.000)
58742,2021,2023,2728500,1222500,72852225,4,4,0,0,4,...,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2728500.000 1222500.000)


In [14]:
# make a sample of statpop cells but inlcude the origin cell in every case
statent_filtered = pd.concat([
    statent_filtered[statent_filtered['id'] == statent_id],
    statent_filtered[statent_filtered['id'] != statent_id].sample(frac=destinations_sample, random_state=0)
    ])

In [15]:
# match statent points to lanegraph nodes
# TODO: do this only once

for mode in [MODE_PRIVATE_CARS, MODE_CYCLING, MODE_FOOT]:

    # match statent points to nearest nodes
    nodes = oxc.nearest_nodes(
        L_modes[mode],
        list(map(lambda geom: geom.x, statent_filtered.geometry)),
        list(map(lambda geom: geom.y, statent_filtered.geometry)),
        return_dist=True
    )

    statent_filtered[[f'closest_node_{mode}', f'egress_{mode}']] = list(zip(*nodes))

statent_filtered

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry,closest_node_private_cars,egress_private_cars,closest_node_cycling,egress_cycling,closest_node_foot,egress_foot
28794,2021,2023,2682900,1249400,68292494,22,0,0,22,47,...,35.955146,15.024609,20.930537,POINT (2682900.000 1249400.000),9432.0,39.012672,28557.0,39.008951,28557.0,39.008951
6655,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,0.000000,0.000000,0.000000,POINT (2653100.000 1239900.000),24514.0,1358.115708,50566.0,241.644861,50566.0,241.644861
5827,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,4.000000,4.000000,0.000000,POINT (2650500.000 1245800.000),53053.0,91.026213,51468.0,91.026213,51468.0,91.026213
38622,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,4.000000,4.000000,4.000000,POINT (2690900.000 1282700.000),49794.0,1371.805307,24747.0,31.243944,24747.0,31.243944
1399,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,6.989673,5.116902,4.000000,POINT (2637700.000 1255500.000),33074.0,51.306988,57357.0,51.306988,57357.0,51.306988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25345,2021,2023,2680800,1248400,68082484,6,0,0,6,7,...,4.000000,4.000000,4.000000,POINT (2680800.000 1248400.000),43106.0,63.246448,75310.0,63.244613,75310.0,63.244613
56523,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,4.000000,0.000000,4.000000,POINT (2714500.000 1258000.000),18198.0,1453.855424,37228.0,1352.131870,37228.0,1352.131870
44234,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,4.000000,4.000000,4.000000,POINT (2696900.000 1228000.000),16756.0,25.600192,4735.0,25.600192,4735.0,25.600192
25275,2021,2023,2680700,1258600,68072586,4,0,0,4,4,...,4.000000,4.000000,4.000000,POINT (2680700.000 1258600.000),7720.0,55.955058,27012.0,55.955058,27012.0,55.955058


In [16]:
tt_matrices = {}

In [17]:
# calculate transit travel time matrix using R5
tt_computer_transit = r5py.TravelTimeMatrixComputer(
    r5_transit_network,
    transport_modes=[r5py.TransportMode.TRANSIT],
    origins=origins,
    destinations=statent_filtered,
    departure=departure_time,
    snap_to_network=True
)
tt_matrices[MODE_TRANSIT] = tt_computer_transit.compute_travel_times()
tt_matrices[MODE_TRANSIT].rename(columns={
    'travel_time': 'travel_time_transit',
    'from_id': 'from_cell',
    'to_id': 'to_cell'
}, inplace=True)
# convert from minutes to seconds
tt_matrices[MODE_TRANSIT]['travel_time_transit'] *= 60
tt_matrices[MODE_TRANSIT]['travel_time_egress_transit'] = 0

tt_matrices[MODE_TRANSIT]

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
C:\Users\lballo\AppData\Local\miniconda3\envs\snman2\lib\site-packages\r5py\r5\base_travel_time_matrix_computer.py:152: RuntimeWarning: Some destination points could not be snapped to the street network
  warnings.warn(


,from_cell,to_cell,travel_time_transit,travel_time_egress_transit
0,68292494,68292494,0.0,0
1,68292494,65312399,NaN,0
2,68292494,65052458,5340.0,0
3,68292494,69092827,5460.0,0
4,68292494,63772555,NaN,0
...,...,...,...,...
2856,68292494,68082484,1620.0,0
2857,68292494,71452580,6900.0,0
2858,68292494,69692280,5040.0,0
2859,68292494,68072586,4200.0,0


In [18]:
# calculate shortest paths to all destinations in networkx
for mode in [MODE_PRIVATE_CARS, MODE_FOOT, MODE_CYCLING]:

    print(mode)

    origin_cell = statent_filtered[statent_filtered['id'] == statent_id]
    origin_node = min(origin_cell[f'closest_node_{mode}'])
    access_cost = min(origin_cell[f'egress_{mode}'])
    print(origin_node, access_cost)
    
    if mode == MODE_PRIVATE_CARS:
        weight = 'matsim_tt_cars'
    else:
        weight = f'cost_{mode}'
    
    # calculate cost to all other nodes
    dijkstra_path_costs = nx.single_source_dijkstra_path_length(L_modes[mode], origin_node, weight=weight)
    
    # convert the dijkstra path costs dict to a dataframe like from R5
    tt_matrices[mode] = pd.DataFrame(list(dijkstra_path_costs.items()), columns=['to_node', f'path_length_{mode}'])
    tt_matrices[mode]['from_node'] = origin_node
    
tt_matrices[MODE_PRIVATE_CARS]

private_cars
9432.0 39.01267193587629
foot
28557.0 39.00895125389578
cycling
28557.0 39.00895125389578


,to_node,path_length_private_cars,from_node
0,9432.0,0.00,9432.0
1,50833.0,17.35,9432.0
2,3701.0,34.07,9432.0
3,45275.0,37.84,9432.0
4,2098.0,39.67,9432.0
...,...,...,...
53888,34240.0,10356.50,9432.0
53889,47627.0,10368.32,9432.0
53890,337.0,10375.25,9432.0
53891,32244.0,11088.15,9432.0


In [19]:
# merge the tt matrix of transit on cell_ids
statent_filtered_2 = pd.merge(
    statent_filtered, tt_matrices[MODE_TRANSIT],
    left_on='id', right_on='to_cell', how='left'
)
statent_filtered_2.drop(columns=['from_cell', 'to_cell'], inplace=True)
statent_filtered_2[f'travel_time_total_{MODE_TRANSIT}'] = statent_filtered_2[f'travel_time_{MODE_TRANSIT}']
statent_filtered_2

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,geometry,closest_node_private_cars,egress_private_cars,closest_node_cycling,egress_cycling,closest_node_foot,egress_foot,travel_time_transit,travel_time_egress_transit,travel_time_total_transit
0,2021,2023,2682900,1249400,68292494,22,0,0,22,47,...,POINT (2682900.000 1249400.000),9432.0,39.012672,28557.0,39.008951,28557.0,39.008951,0.0,0.0,0.0
1,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,POINT (2653100.000 1239900.000),24514.0,1358.115708,50566.0,241.644861,50566.0,241.644861,NaN,0.0,NaN
2,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,POINT (2650500.000 1245800.000),53053.0,91.026213,51468.0,91.026213,51468.0,91.026213,5340.0,0.0,5340.0
3,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,POINT (2690900.000 1282700.000),49794.0,1371.805307,24747.0,31.243944,24747.0,31.243944,5460.0,0.0,5460.0
4,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,POINT (2637700.000 1255500.000),33074.0,51.306988,57357.0,51.306988,57357.0,51.306988,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2021,2023,2680800,1248400,68082484,6,0,0,6,7,...,POINT (2680800.000 1248400.000),43106.0,63.246448,75310.0,63.244613,75310.0,63.244613,1620.0,0.0,1620.0
2934,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,POINT (2714500.000 1258000.000),18198.0,1453.855424,37228.0,1352.131870,37228.0,1352.131870,6900.0,0.0,6900.0
2935,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,POINT (2696900.000 1228000.000),16756.0,25.600192,4735.0,25.600192,4735.0,25.600192,5040.0,0.0,5040.0
2936,2021,2023,2680700,1258600,68072586,4,0,0,4,4,...,POINT (2680700.000 1258600.000),7720.0,55.955058,27012.0,55.955058,27012.0,55.955058,4200.0,0.0,4200.0


In [20]:
# merge the other tt matrices on node ids
for mode in [MODE_PRIVATE_CARS, MODE_FOOT, MODE_CYCLING]:

    # merge the travel time matrices with the statent dataset
    # such that every destination has the cost of reaching it
    statent_filtered_2 = pd.merge(
        statent_filtered_2, tt_matrices[mode],
        left_on=f'closest_node_{mode}', right_on='to_node', how='left'
    )
    statent_filtered_2.drop(columns=['from_node', 'to_node'], inplace=True)
    
    origin_cell = statent_filtered[statent_filtered['id'] == statent_id]
    access_cost = min(origin_cell[f'egress_{mode}'])
    
    if mode == MODE_FOOT:
        speed_factor = walking_speed_kmh / 3.6
    elif mode == MODE_CYCLING:
        speed_factor = cycling_speed_kmh / 3.6
    else:
        speed_factor = 1
    
    # calculate total travel time by adding access and egress cost and considering speed factors
    statent_filtered_2[f'travel_time_{mode}'] = (
            (statent_filtered_2[f'path_length_{mode}'] / speed_factor)
    )
    
    statent_filtered_2[f'travel_time_access_egress_{mode}'] = (
            + ((access_cost * access_egress_detour_factor) / (walking_speed_kmh / 3.6))
            + ((statent_filtered_2[f'egress_{mode}'] * access_egress_detour_factor) / (walking_speed_kmh / 3.6))
    )

statent_filtered_2

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,travel_time_total_transit,path_length_private_cars,travel_time_private_cars,travel_time_access_egress_private_cars,path_length_foot,travel_time_foot,travel_time_access_egress_foot,path_length_cycling,travel_time_cycling,travel_time_access_egress_cycling
0,2021,2023,2682900,1249400,68292494,22,0,0,22,47,...,0.0,0.00,0.00,116.455737,0.000000,0.000000,116.444631,0.000000,0.000000,116.444631
1,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,NaN,3081.30,3081.30,2085.266239,45962.234831,34300.175247,418.886288,37177.173083,7435.434617,418.886288
2,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,5340.0,2962.74,2962.74,194.087888,42963.660513,32062.433219,194.082335,31321.094135,6264.218827,194.082335
3,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,5460.0,2322.08,2322.08,2105.698476,44340.542136,33089.956818,104.855068,30145.297967,6029.059593,104.855068
4,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,NaN,2846.17,2846.17,134.805463,57942.862590,43240.942232,134.799910,43917.811288,8783.562258,134.799910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2021,2023,2680800,1248400,68082484,6,0,0,6,7,...,1620.0,1239.26,1239.26,152.625553,3516.126844,2623.975257,152.617260,1920.594425,384.118885,152.617260
2934,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,6900.0,2576.54,2576.54,2228.161337,40581.541607,30284.732543,2076.329584,23840.792101,4768.158420,2076.329584
2935,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,5040.0,2015.54,2015.54,96.437110,29293.709964,21860.977585,96.431557,16363.299860,3272.659972,96.431557
2936,2021,2023,2680700,1258600,68072586,4,0,0,4,4,...,4200.0,1335.86,1335.86,141.742881,11412.909507,8517.096647,141.737328,6655.627093,1331.125419,141.737328


In [21]:
statent_filtered_2['from_cell'] = statent_id
statent_filtered_2

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,path_length_private_cars,travel_time_private_cars,travel_time_access_egress_private_cars,path_length_foot,travel_time_foot,travel_time_access_egress_foot,path_length_cycling,travel_time_cycling,travel_time_access_egress_cycling,from_cell
0,2021,2023,2682900,1249400,68292494,22,0,0,22,47,...,0.00,0.00,116.455737,0.000000,0.000000,116.444631,0.000000,0.000000,116.444631,68292494
1,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,3081.30,3081.30,2085.266239,45962.234831,34300.175247,418.886288,37177.173083,7435.434617,418.886288,68292494
2,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,2962.74,2962.74,194.087888,42963.660513,32062.433219,194.082335,31321.094135,6264.218827,194.082335,68292494
3,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,2322.08,2322.08,2105.698476,44340.542136,33089.956818,104.855068,30145.297967,6029.059593,104.855068,68292494
4,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,2846.17,2846.17,134.805463,57942.862590,43240.942232,134.799910,43917.811288,8783.562258,134.799910,68292494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2021,2023,2680800,1248400,68082484,6,0,0,6,7,...,1239.26,1239.26,152.625553,3516.126844,2623.975257,152.617260,1920.594425,384.118885,152.617260,68292494
2934,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,2576.54,2576.54,2228.161337,40581.541607,30284.732543,2076.329584,23840.792101,4768.158420,2076.329584,68292494
2935,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,2015.54,2015.54,96.437110,29293.709964,21860.977585,96.431557,16363.299860,3272.659972,96.431557,68292494
2936,2021,2023,2680700,1258600,68072586,4,0,0,4,4,...,1335.86,1335.86,141.742881,11412.909507,8517.096647,141.737328,6655.627093,1331.125419,141.737328,68292494


In [22]:
# add euclidean distance

origin_cell = statent_filtered[statent_filtered['id'] == statent_id]
origin_geometry = min(origin_cell.geometry)
statent_filtered_2['euclidean_distance'] = statent_filtered_2.apply(
    lambda row: shp.distance(origin_geometry, row.geometry),
    axis=1
)
statent_filtered_2

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,travel_time_private_cars,travel_time_access_egress_private_cars,path_length_foot,travel_time_foot,travel_time_access_egress_foot,path_length_cycling,travel_time_cycling,travel_time_access_egress_cycling,from_cell,euclidean_distance
0,2021,2023,2682900,1249400,68292494,22,0,0,22,47,...,0.00,116.455737,0.000000,0.000000,116.444631,0.000000,0.000000,116.444631,68292494,0.000000
1,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,3081.30,2085.266239,45962.234831,34300.175247,418.886288,37177.173083,7435.434617,418.886288,68292494,31277.627787
2,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,2962.74,194.087888,42963.660513,32062.433219,194.082335,31321.094135,6264.218827,194.082335,68292494,32599.386497
3,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,2322.08,2105.698476,44340.542136,33089.956818,104.855068,30145.297967,6029.059593,104.855068,68292494,34247.481659
4,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,2846.17,134.805463,57942.862590,43240.942232,134.799910,43917.811288,8783.562258,134.799910,68292494,45609.757728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2021,2023,2680800,1248400,68082484,6,0,0,6,7,...,1239.26,152.625553,3516.126844,2623.975257,152.617260,1920.594425,384.118885,152.617260,68292494,2325.940670
2934,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,2576.54,2228.161337,40581.541607,30284.732543,2076.329584,23840.792101,4768.158420,2076.329584,68292494,32749.351139
2935,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,2015.54,96.437110,29293.709964,21860.977585,96.431557,16363.299860,3272.659972,96.431557,68292494,25572.641631
2936,2021,2023,2680700,1258600,68072586,4,0,0,4,4,...,1335.86,141.742881,11412.909507,8517.096647,141.737328,6655.627093,1331.125419,141.737328,68292494,9459.386872


In [23]:
destinations_with_cost = pd.merge(
    statpop_reduced.reset_index(),
    statent_filtered_2.reset_index()[[
        'id', 'VOLLZEITAEQ_TOTAL',
        'euclidean_distance',
        'travel_time_foot',
        'travel_time_cycling',
        'travel_time_transit',
        'travel_time_private_cars',
        'travel_time_access_egress_foot',
        'travel_time_access_egress_cycling',
        'travel_time_access_egress_private_cars',
        'path_length_private_cars',
        'path_length_cycling',
        'path_length_foot',
        'closest_node_private_cars',
        'from_cell',
        'geometry'
    ]].rename(columns={'geometry': 'destination_geometry'}),
    how='left', left_on='statent_id', right_on='from_cell')
destinations_with_cost

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,travel_time_private_cars,travel_time_access_egress_foot,travel_time_access_egress_cycling,travel_time_access_egress_private_cars,path_length_private_cars,path_length_cycling,path_length_foot,closest_node_private_cars,from_cell,destination_geometry
0,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),68292494,35.955146,...,0.00,116.444631,116.444631,116.455737,0.00,0.000000,0.000000,9432.0,68292494,POINT (2682900.000 1249400.000)
1,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65312399,4.000000,...,3081.30,418.886288,418.886288,2085.266239,3081.30,37177.173083,45962.234831,24514.0,68292494,POINT (2653100.000 1239900.000)
2,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65052458,4.000000,...,2962.74,194.082335,194.082335,194.087888,2962.74,31321.094135,42963.660513,53053.0,68292494,POINT (2650500.000 1245800.000)
3,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),69092827,4.000000,...,2322.08,104.855068,104.855068,2105.698476,2322.08,30145.297967,44340.542136,49794.0,68292494,POINT (2690900.000 1282700.000)
4,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),63772555,6.989673,...,2846.17,134.799910,134.799910,134.805463,2846.17,43917.811288,57942.862590,33074.0,68292494,POINT (2637700.000 1255500.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38189,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68082484,4.000000,...,1239.26,152.617260,152.617260,152.625553,1239.26,1920.594425,3516.126844,43106.0,68292494,POINT (2680800.000 1248400.000)
38190,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),71452580,4.000000,...,2576.54,2076.329584,2076.329584,2228.161337,2576.54,23840.792101,40581.541607,18198.0,68292494,POINT (2714500.000 1258000.000)
38191,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),69692280,4.000000,...,2015.54,96.431557,96.431557,96.437110,2015.54,16363.299860,29293.709964,16756.0,68292494,POINT (2696900.000 1228000.000)
38192,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68072586,4.000000,...,1335.86,141.737328,141.737328,141.742881,1335.86,6655.627093,11412.909507,7720.0,68292494,POINT (2680700.000 1258600.000)


In [24]:
# ensure minimum travel time and euclidian distance
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    destinations_with_cost[f'travel_time_{mode}'] = np.maximum(destinations_with_cost[f'travel_time_{mode}'], min_travel_time)

destinations_with_cost['euclidean_distance'] = np.maximum(destinations_with_cost['euclidean_distance'], min_euclidian_distance)

In [25]:
#run the mode choice model to calculate each person's generalized cost to each statent destination
def calculate_behavioral_cost(
    euclidean_distance,
    travel_time_private_cars=np.inf, travel_time_access_egress_private_cars=np.inf, path_length_private_cars=np.inf,
    travel_time_transit=np.inf,
    travel_time_cycling=np.inf, travel_time_access_egress_cycling=np.inf,
    travel_time_foot=np.inf, travel_time_access_egress_foot=np.inf,
    age=None
):
    """
    Calculates individual generalized cost based on the personal properties and mode choice.
    Based on mode choice model implemented in eqasim.
    
    Returns a tuple of mode specific choice probabilities (dict) and the resulting behavioral travel time (float)
    """
    
    U = {}
    
    X_dist = euclidean_distance / 1000
    Mi_hhIncome = 12260
    X_hhIncome = 10000
    X_work = 0.5
    X_city_center = 0.2
    Mi_dist = 39
    Lambda_distTT = 0.1147
    Beta_cost = -0.088
    Lambda_dist_cost = -0.2209
    Lambda_hhIncome = -0.8169
    
    X_IVT_car = travel_time_private_cars / 60
    X_AET_car = travel_time_access_egress_private_cars / 60
    X_cost_car = 0.26 * euclidean_distance / 1000 * 1.5
    Alpha_car = -0.8
    Beta_TT_car = -0.0192
    Beta_TT_walk = -0.0457
    Beta_work_car = -1.1606
    Beta_city_center_car = -0.4590
    
    U[MODE_PRIVATE_CARS] = (
        Alpha_car
        + Beta_TT_car * X_IVT_car * (X_dist / Mi_dist) ** Lambda_distTT
        + Beta_TT_walk * X_AET_car
        + Beta_cost * (X_dist / Mi_dist) ** Lambda_dist_cost * X_cost_car * (X_hhIncome / Mi_hhIncome) ** Lambda_hhIncome
        + Beta_work_car * X_work
        + Beta_city_center_car * X_city_center
    )
    
    X_railTT = travel_time_transit * 0.8 / 60  # surrogate
    X_busTT = travel_time_transit * 0.2 / 60 # surrogate
    X_AET_PT = 0 # = 0 because R5 includes it in the travel time
    X_waiting_time = 5 # surrogate
    X_number_of_connections = travel_time_transit / 60 / 20
    X_cost_PT = min(2.7, euclidean_distance / 1000 * 0.6)
    X_headway = 10 # surrogate
    Alpha_PT = 0
    Beta_railTT = -0.0072
    Beta_busTT = -0.0124
    Beta_AET_PT = -0.0142
    Beta_wait = -0.0124
    Beta_lineSwitch = -0.17
    Beta_headway = -0.0301
    Beta_OVGK = -0.8 # surrogate    
    
    U[MODE_TRANSIT] = (
        Alpha_PT
        + (Beta_railTT * X_railTT + Beta_busTT * X_busTT) * (X_dist / Mi_dist) ** Lambda_distTT
        + Beta_AET_PT * X_AET_PT
        + Beta_wait * X_waiting_time
        + Beta_lineSwitch * X_number_of_connections
        + Beta_cost * (X_dist / Mi_dist) ** Lambda_dist_cost * X_cost_PT * (X_hhIncome / Mi_hhIncome) ** Lambda_hhIncome
        + Beta_headway * X_headway
        + Beta_OVGK
    )
    
    X_bikeTT = travel_time_cycling / 60 + travel_time_access_egress_cycling / 60
    X_age = age
    Alpha_bike = -0.1258
    Beta_TT_bike = -0.1258
    Beta_age_60_plus_bike = -2.6588
    
    U[MODE_CYCLING]= (
        Alpha_bike
        + Beta_TT_bike * X_bikeTT + (X_dist / Mi_dist) ** Lambda_distTT
        + Beta_age_60_plus_bike * (X_age >= 60)
    )
    
    X_walkTT = travel_time_foot / 60 + travel_time_access_egress_foot / 60
    Alpha_walk = 0.5903
    Beta_TT_walk = -0.0457
    Theta_threshold_walkTT = 120
    
    U[MODE_FOOT] = (
        Alpha_walk
        + Beta_TT_walk * X_walkTT * (X_dist / Mi_dist) ** Lambda_distTT
        + (1 - 100 ** (X_walkTT / Theta_threshold_walkTT))
    )

    # replace nan with -inf for non-existent options (=infinite cost)
    U = {mode: -np.inf if np.isnan(U_mode) else U_mode for mode, U_mode in U.items()}

    # mode-specific choice probabilities
    denominator = sum([np.exp(U_mode) for U_mode in U.values()])
    if denominator != 0:
        
        P = {mode: np.exp(U_mode) / denominator for mode, U_mode in U.items()}
        #print(U)
        
        # mode-specific cost
        C = {mode: -U_mode for mode, U_mode in U.items()}
        
        # weighted cost
        C_weighted = sum([C[mode] * P[mode] if not np.isinf(C[mode]) else 0 for mode in P.keys()])
    
    else:
        
        P = {mode: None for mode, U_mode in U.items()}
        C = {mode: np.inf for mode, U_mode in U.items()}
        C_weighted = np.inf
        
    return P, C, C_weighted
        

destinations_with_cost[['choice_probabilities', 'cost', 'weighted_cost']] = destinations_with_cost.apply(
    lambda row: calculate_behavioral_cost(
        **row[[
            'euclidean_distance',
            'travel_time_private_cars', 'travel_time_access_egress_private_cars', 'path_length_private_cars',
            'travel_time_transit',
            'travel_time_cycling', 'travel_time_access_egress_cycling',
            'travel_time_foot', 'travel_time_access_egress_foot',
            'age'
        ]]
    ),
    axis=1,
    result_type='expand'
)

destinations_with_cost

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,travel_time_access_egress_private_cars,path_length_private_cars,path_length_cycling,path_length_foot,closest_node_private_cars,from_cell,destination_geometry,choice_probabilities,cost,weighted_cost
0,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),68292494,35.955146,...,116.455737,0.00,0.000000,0.000000,9432.0,68292494,POINT (2682900.000 1249400.000),"{'private_cars': 0.08848461978031191, 'transit...","{'private_cars': 1.6243688073055653, 'transit'...",0.376349
1,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65312399,4.000000,...,2085.266239,3081.30,37177.173083,45962.234831,24514.0,68292494,POINT (2653100.000 1239900.000),"{'private_cars': 0.9999651962209564, 'transit'...","{'private_cars': 5.352934986516798, 'transit':...",5.353292
2,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65052458,4.000000,...,194.087888,2962.74,31321.094135,42963.660513,53053.0,68292494,POINT (2650500.000 1245800.000),"{'private_cars': 0.2701859855987031, 'transit'...","{'private_cars': 3.923522029545871, 'transit':...",3.198775
3,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),69092827,4.000000,...,2105.698476,2322.08,30145.297967,44340.542136,49794.0,68292494,POINT (2690900.000 1282700.000),"{'private_cars': 0.09340526746979325, 'transit...","{'private_cars': 5.236677880462104, 'transit':...",3.177298
4,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),63772555,6.989673,...,134.805463,2846.17,43917.811288,57942.862590,33074.0,68292494,POINT (2637700.000 1255500.000),"{'private_cars': 0.9999986541625661, 'transit'...","{'private_cars': 4.288020272155529, 'transit':...",4.288038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38189,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68082484,4.000000,...,152.625553,1239.26,1920.594425,3516.126844,43106.0,68292494,POINT (2680800.000 1248400.000),"{'private_cars': 0.1456361213848126, 'transit'...","{'private_cars': 2.0510986972813154, 'transit'...",1.003746
38190,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),71452580,4.000000,...,2228.161337,2576.54,23840.792101,40581.541607,18198.0,68292494,POINT (2714500.000 1258000.000),"{'private_cars': 0.11960715584277344, 'transit...","{'private_cars': 5.357089362578564, 'transit':...",3.600077
38191,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),69692280,4.000000,...,96.437110,2015.54,16363.299860,29293.709964,16756.0,68292494,POINT (2696900.000 1228000.000),"{'private_cars': 0.3807169721577951, 'transit'...","{'private_cars': 3.2979350224300275, 'transit'...",3.085481
38192,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68072586,4.000000,...,141.742881,1335.86,6655.627093,11412.909507,7720.0,68292494,POINT (2680700.000 1258600.000),"{'private_cars': 0.3380699990791281, 'transit'...","{'private_cars': 2.4677490394631656, 'transit'...",2.475911


In [26]:
destinations_with_cost = pd.concat([
    destinations_with_cost,
    pd.json_normalize(destinations_with_cost['choice_probabilities']).add_prefix('p_'),
    pd.json_normalize(destinations_with_cost['cost']).add_prefix('c_'),
    ],
    axis=1
)

# calculate total accessibility contribution using cost function and destination utility
destinations_with_cost['accessibility_contribution'] = (
    (destinations_with_cost['weighted_cost'] ** -0.7) 
    * destinations_with_cost['VOLLZEITAEQ_TOTAL']
)

destinations_with_cost

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,weighted_cost,p_private_cars,p_transit,p_cycling,p_foot,c_private_cars,c_transit,c_cycling,c_foot,accessibility_contribution
0,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),68292494,35.955146,...,0.376349,0.088485,0.128717,0.273879,5.089190e-01,1.624369,1.249579,0.494511,-1.250914e-01,71.260292
1,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65312399,4.000000,...,5.353292,0.999965,0.000000,0.000035,0.000000e+00,5.352935,inf,15.618685,4.407413e+09,1.236020
2,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65052458,4.000000,...,3.198775,0.270186,0.729772,0.000042,0.000000e+00,3.923522,2.929901,12.687057,9.123116e+08,1.772439
3,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),69092827,4.000000,...,3.177298,0.093405,0.906487,0.000108,0.000000e+00,5.236678,2.964049,12.001369,1.662574e+09,1.780817
4,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),63772555,6.989673,...,4.288038,0.999999,0.000000,0.000001,0.000000e+00,4.288020,inf,17.806513,1.118967e+12,2.522759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38189,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68082484,4.000000,...,1.003746,0.145636,0.182784,0.668305,3.274810e-03,2.051099,1.823906,0.527465,5.845950e+00,3.989543
38190,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),71452580,4.000000,...,3.600077,0.119607,0.880358,0.000035,0.000000e+00,5.357089,3.360973,13.496246,9.754027e+08,1.631705
38191,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),69692280,4.000000,...,3.085481,0.380717,0.599136,0.020147,0.000000e+00,3.297935,2.844503,6.236917,1.256855e+06,1.817749
38192,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68072586,4.000000,...,2.475911,0.338070,0.286852,0.375078,2.784678e-112,2.467749,2.632036,2.363868,2.582486e+02,2.120535


In [27]:
# calculate accessibility contribution for every mode separately
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    destinations_with_cost[f'accessibility_contribution_{mode}'] = (
        (destinations_with_cost[f'c_{mode}'] ** -0.7)
        * destinations_with_cost['VOLLZEITAEQ_TOTAL']
    )
    
destinations_with_cost

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,p_foot,c_private_cars,c_transit,c_cycling,c_foot,accessibility_contribution,accessibility_contribution_cycling,accessibility_contribution_private_cars,accessibility_contribution_transit,accessibility_contribution_foot
0,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),68292494,35.955146,...,5.089190e-01,1.624369,1.249579,0.494511,-1.250914e-01,71.260292,58.862394,25.602467,30.762841,NaN
1,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65312399,4.000000,...,0.000000e+00,5.352935,inf,15.618685,4.407413e+09,1.236020,0.584129,1.236077,0.000000,7.097924e-07
2,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),65052458,4.000000,...,0.000000e+00,3.923522,2.929901,12.687057,9.123116e+08,1.772439,0.675628,1.536338,1.884790,2.137764e-06
3,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),69092827,4.000000,...,0.000000e+00,5.236678,2.964049,12.001369,1.662574e+09,1.780817,0.702423,1.255223,1.869564,1.404472e-06
4,7722724,40,1,1,-2,-2,68292494.0,POINT (2682915.001 1249401.001),63772555,6.989673,...,0.000000e+00,4.288020,inf,17.806513,1.118967e+12,2.522759,0.931219,2.522767,0.000000,2.572082e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38189,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68082484,4.000000,...,3.274810e-03,2.051099,1.823906,0.527465,5.845950e+00,3.989543,6.259282,2.419187,2.626384,1.162148e+00
38190,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),71452580,4.000000,...,0.000000e+00,5.357089,3.360973,13.496246,9.754027e+08,1.631705,0.647010,1.235406,1.712121,2.040005e-06
38191,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),69692280,4.000000,...,0.000000e+00,3.297935,2.844503,6.236917,1.256855e+06,1.817749,1.110660,1.734964,1.924223,2.150603e-04
38192,5296966,53,1,2,-2,-2,68292494.0,POINT (2682857.001 1249408.001),68072586,4.000000,...,2.784678e-112,2.467749,2.632036,2.363868,2.582486e+02,2.120535,2.190401,2.125442,2.031682,8.196593e-02


In [28]:
# for each person, sum the accessibility contributions across all destinations
accessibility = destinations_with_cost.groupby('record').agg({
    'age': 'first',
    'sex': 'first',
    'maritalstatus': 'first',
    'residencepermit': 'first',
    'residentpermit': 'first',
    'statent_id': 'first',
    'accessibility_contribution': 'sum',
    'accessibility_contribution_cycling': 'sum',
    'accessibility_contribution_foot': 'sum',
    'accessibility_contribution_private_cars': 'sum',
    'accessibility_contribution_transit': 'sum',
    'geometry': 'first'
})

accessibility.rename(columns={
    'accessibility_contribution': 'accessibility',
    'accessibility_contribution_cycling': 'accessibility_cycling',
    'accessibility_contribution_foot': 'accessibility_foot',
    'accessibility_contribution_private_cars': 'accessibility_private_cars',
    'accessibility_contribution_transit': 'accessibility_transit',
}, inplace=True)

accessibility.reset_index(inplace=True)
accessibility = gpd.GeoDataFrame(accessibility, crs=2056)
accessibility

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility,accessibility_cycling,accessibility_foot,accessibility_private_cars,accessibility_transit,geometry
0,3434419,34,2,1,201,2,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682894.001 1249408.001)
1,4128950,52,2,4,301,3,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682892.001 1249431.001)
2,5044070,60,2,2,301,3,68292494.0,35620.618541,20390.788920,7078.411662,33844.767229,36902.413853,POINT (2682915.001 1249401.001)
3,5296966,53,1,2,-2,-2,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682857.001 1249408.001)
4,5778759,38,1,2,-2,-2,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682938.001 1249402.001)
5,6630206,39,2,2,-2,-2,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682856.001 1249430.001)
6,7234678,64,1,2,-2,-2,68292494.0,35620.618541,20390.788920,7078.411662,33844.767229,36902.413853,POINT (2682932.001 1249371.001)
7,7259080,39,2,1,-2,-2,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682938.001 1249356.001)
8,7722724,40,1,1,-2,-2,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682915.001 1249401.001)
9,8324543,33,2,1,-2,-2,68292494.0,43027.311973,48373.381405,7078.411662,33844.767229,36902.413853,POINT (2682938.001 1249356.001)


In [29]:
if 0:
    snman.io.export_gdf(
        accessibility,
        os.path.join(outputs_path, f'accessibility_{STATE}.gpkg')
    )

In [30]:
accessibility_contributions = copy.deepcopy(destinations_with_cost)
accessibility_contributions['geometry'] = accessibility_contributions.apply(
    lambda row: shp.LineString([row['geometry'], row['destination_geometry']]),
    axis=1
)
accessibility_contributions['geometry'] = accessibility_contributions['destination_geometry']
accessibility_contributions.drop(columns=['destination_geometry'], inplace=True)

snman.io.export_gdf(
    accessibility_contributions,
    os.path.join(outputs_path, f'accessibility_contributions_{STATE}.gpkg')
)

accessibility_contributions

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,p_foot,c_private_cars,c_transit,c_cycling,c_foot,accessibility_contribution,accessibility_contribution_cycling,accessibility_contribution_private_cars,accessibility_contribution_transit,accessibility_contribution_foot
0,7722724,40,1,1,-2,-2,68292494.0,POINT (2682900.000 1249400.000),68292494,35.955146,...,5.089190e-01,1.624369,1.249579,0.494511,-1.250914e-01,71.260292,58.862394,25.602467,30.762841,NaN
1,7722724,40,1,1,-2,-2,68292494.0,POINT (2653100.000 1239900.000),65312399,4.000000,...,0.000000e+00,5.352935,inf,15.618685,4.407413e+09,1.236020,0.584129,1.236077,0.000000,7.097924e-07
2,7722724,40,1,1,-2,-2,68292494.0,POINT (2650500.000 1245800.000),65052458,4.000000,...,0.000000e+00,3.923522,2.929901,12.687057,9.123116e+08,1.772439,0.675628,1.536338,1.884790,2.137764e-06
3,7722724,40,1,1,-2,-2,68292494.0,POINT (2690900.000 1282700.000),69092827,4.000000,...,0.000000e+00,5.236678,2.964049,12.001369,1.662574e+09,1.780817,0.702423,1.255223,1.869564,1.404472e-06
4,7722724,40,1,1,-2,-2,68292494.0,POINT (2637700.000 1255500.000),63772555,6.989673,...,0.000000e+00,4.288020,inf,17.806513,1.118967e+12,2.522759,0.931219,2.522767,0.000000,2.572082e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38189,5296966,53,1,2,-2,-2,68292494.0,POINT (2680800.000 1248400.000),68082484,4.000000,...,3.274810e-03,2.051099,1.823906,0.527465,5.845950e+00,3.989543,6.259282,2.419187,2.626384,1.162148e+00
38190,5296966,53,1,2,-2,-2,68292494.0,POINT (2714500.000 1258000.000),71452580,4.000000,...,0.000000e+00,5.357089,3.360973,13.496246,9.754027e+08,1.631705,0.647010,1.235406,1.712121,2.040005e-06
38191,5296966,53,1,2,-2,-2,68292494.0,POINT (2696900.000 1228000.000),69692280,4.000000,...,0.000000e+00,3.297935,2.844503,6.236917,1.256855e+06,1.817749,1.110660,1.734964,1.924223,2.150603e-04
38192,5296966,53,1,2,-2,-2,68292494.0,POINT (2680700.000 1258600.000),68072586,4.000000,...,2.784678e-112,2.467749,2.632036,2.363868,2.582486e+02,2.120535,2.190401,2.125442,2.031682,8.196593e-02


In [31]:
STATE

'after'

In [32]:
# testbed for point to point routing

# node IDs
#origin, destination = (1756, 51785) # cross-city before
origin, destination = (50608, 1776) # cross-city after
#origin, destination = (50608, 35180)
#weight = 'matsim_tt_cars'
weight = 'cost_cycling'

L = L_modes[MODE_CYCLING]
route = nx.shortest_path(L, origin, destination, weight=weight)

uvk_and_order = {}
for u, v in zip(route[:-1], route[1:]):
    k, data = min(L.get_edge_data(u, v).items(), key=lambda e: e[1][weight] + (e[1]['lane'].lanetype == LANETYPE_MOTORIZED))
    uvk_and_order[(u,v,k)] = len(uvk_and_order)
    
M = L.edge_subgraph(uvk_and_order.keys())
nx.set_edge_attributes(M, uvk_and_order, 'order_in_path')
M = snman.lane_graph.LaneGraph(M)

edges = oxc.graph_to_gdfs(M, nodes=False)
print(edges.groupby('lanetype').agg({weight: 'sum', 'length': 'sum'}))
print(edges.agg({weight: 'sum', 'length': 'sum'}))

snman.io.export_lane_geometries(
    M,
    os.path.join(outputs_path, f'path_edges.gpkg'),
    os.path.join(outputs_path, f'path_nodes.gpkg'),
    scaling=30
)

          cost_cycling        length
lanetype                            
L          8199.917400  16734.525306
M         13449.929842  13449.929842
S           149.101000    149.101000
X         11423.953557  23314.190932
cost_cycling    33222.901799
length          53647.747080
dtype: float64


In [33]:
STATE

'after'